In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load data
data = np.load('image_data.npz')
X = data['images']  # Shape: (num_samples, 64, 64, 3)
y = data['labels']

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  
y_categorical = to_categorical(y, num_classes=23)  # One-hot encoding for 23 classes

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Resize images to match Xception’s requirement (71x71)
X_train_resized = tf.image.resize(X_train, (71, 71)).numpy()
X_test_resized = tf.image.resize(X_test, (71, 71)).numpy()

# Normalize images
X_train_resized = X_train_resized.astype(np.float32) / 255.0
X_test_resized = X_test_resized.astype(np.float32) / 255.0

# Function to create the Xception model with chosen best parameters
def create_model():
    base_model = Xception(weights='imagenet', include_top=False, input_shape=(71, 71, 3))
    x = GlobalAveragePooling2D()(base_model.output)  # Better feature extraction
    x = Dense(128, activation='relu')(x)  # 128 dense units
    output = Dense(23, activation='softmax')(x)  # 23 classes
    model = Model(inputs=base_model.input, outputs=output)
    
    model.compile(optimizer=Adam(learning_rate=0.0001),  # A smaller learning rate for fine-tuning
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create and train the model
model = create_model()

# Train the model with 32 batch size and 10 epochs
model.fit(X_train_resized, y_train, batch_size=32, epochs=10, validation_data=(X_test_resized, y_test))


Epoch 1/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 786s 2s/step - accuracy: 0.1800 - loss: 2.8175 - val_accuracy: 0.1442 - val_loss: 2.9079
Epoch 2/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 789s 2s/step - accuracy: 0.4353 - loss: 1.9100 - val_accuracy: 0.4133 - val_loss: 2.0450
Epoch 3/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 658s 1s/step - accuracy: 0.6369 - loss: 1.2442 - val_accuracy: 0.4568 - val_loss: 1.9535
Epoch 4/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 653s 1s/step - accuracy: 0.7966 - loss: 0.7324 - val_accuracy: 0.4627 - val_loss: 2.1616
Epoch 5/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.8848 - loss: 0.4036 - val_accuracy: 0.4721 - val_loss: 2.2730
Epoch 6/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 653s 1s/step - accuracy: 0.9316 - loss: 0.2416 - val_accuracy: 0.4760 - val_loss: 2.3710
Epoch 7/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 650s 1s/step - accuracy: 0.9480 - loss: 0.1708 - val_accuracy: 0.4532 - val_loss: 2.7848
Epoch 8/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.9529 - loss: 0.1480 - val_accu

In [32]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4%}")


123/123 ━━━━━━━━━━━━━━━━━━━━ 20s 159ms/step - accuracy: 0.4808 - loss: 2.7886
Test Loss: 2.7830
Test Accuracy: 48.2618%
